In [1]:
import pandas as pd
import yaml
import csv
import json
import time

import torch
import torch.nn as nn
from torch import nn, optim, Tensor
import torch.nn.functional as F

import numpy as np
from typing import List, Tuple, Any, Optional, Dict

from training.data import Instance, Data, Dictionary, triples_to_indices
from models.encoders import EncoderLSTM

import time

Triple = Fact = Tuple[str, str, str]
Story = List[Fact]

In [10]:
data = Data()
dictionary = Dictionary(data)
story, target = data.train[10000].story, data.train[0].target
s, p, o = target
print(f'Number of entities {dictionary.num_entities}')
print(f'Example story: {story}')
print(f'Example target: {target}')

entity_embeddings = nn.Embedding(dictionary.num_entities, 100, sparse=True)
relation_embeddings = nn.Embedding(dictionary.num_relations, 100, sparse=True)
print(f'entity embeddings shape: {entity_embeddings.weight.shape}')

indices, ent, rel = triples_to_indices(dictionary, story)

Number of entities 78
Example story: [('Paul', 'mother', 'Michelle'), ('Michelle', 'brother', 'Elliott'), ('Elliott', 'sister', 'Valerie')]
Example target: ('Donald', 'aunt', 'Dorothy')
entity embeddings shape: torch.Size([78, 100])


In [5]:
model = EncoderLSTM(hidden_size = 200,
                    entity_embeddings = entity_embeddings,
                    relation_embeddings = relation_embeddings)

In [6]:
out = model.forward(indices, None);